In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# !pip install --user --upgrade pip
# !pip install --user -q tensorflow==2.0.0-alpha0

import tensorflow as tf

# tf.enable_eager_execution()
print("Eager Execution: {}".format(tf.executing_eagerly()))
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution is: {}".format(tf.executing_eagerly()))
print("Keras version: {}".format(tf.keras.__version__))

# Install a temporary patch to enable a few extra TF 2.0 upgrades. This piece will be removed soon.

from tensorflow.python.ops import control_flow_util
control_flow_util.ENABLE_CONTROL_FLOW_V2 = True

#@tf.function
var = tf.Variable([3, 3])

if tf.config.list_physical_devices('GPU'):
    print('Running on GPU')
    print('GPU #0?')
    print(var.device.endswith('GPU:0'))
else: 
    print('Running on CPU')

/root/miniconda3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/miniconda3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/miniconda3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/miniconda3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarnin

Eager Execution: True
TensorFlow version: 2.0.0-beta1
Eager execution is: True
Keras version: 2.2.4-tf
Running on CPU


In [2]:
mnist = tf.keras.datasets.mnist

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [4]:
# Normalizing Data

x_train, y_train = tf.cast(x_train/255., dtype=tf.float32), tf.cast(y_train/255., tf.float32)
x_test, y_test = tf.cast(x_test/255., dtype=tf.float32), tf.cast(y_test/255., tf.float32)

In [18]:
batch_size = 32
epochs = 100
num_classes = 10

In [6]:
# Using Dataset to represent an input Pipline

# https://www.tensorflow.org/guide/datasets#consuming_numpy_arrays
ds_train = tf.data.Dataset.from_tensor_slices( (x_train, y_train) ).batch(32).shuffle(8888)

In [7]:
type(ds_train)

tensorflow.python.data.ops.dataset_ops.ShuffleDataset

In [8]:
ds_train = ds_train.map(lambda x, y: (tf.image.random_flip_left_right(x), y))   # 隨機水平翻轉影像，加強資料差異, 但標籤 y 不變

In [9]:
ds_train = ds_train.repeat()  #無限重複

In [10]:
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size).shuffle(8888)
ds_test = ds_test.repeat()

In [11]:
model = tf.keras.models.Sequential([
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation=tf.nn.relu),
 tf.keras.layers.Dropout(0.3),
 tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax)
])

In [12]:
optimizer = tf.keras.optimizers.Adam()

In [13]:
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [14]:
model.fit(ds_train, epochs=epochs )

ValueError: When passing an infinitely repeating dataset, you must specify the `steps_per_epoch` argument.

In [15]:
steps_per_epoch = len(x_train) // batch_size

model.fit(ds_train, epochs=epochs, steps_per_epoch=steps_per_epoch)

Epoch 1/10


W0826 08:22:15.498340 139871233808192 deprecation.py:323] From /root/miniconda3/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


1875/1875 [==============================] - 23s 12ms/step - loss: 0.0027 - accuracy: 0.0992
Epoch 2/10
1875/1875 [==============================] - 22s 12ms/step - loss: 2.3112e-06 - accuracy: 0.0967
Epoch 3/10
1875/1875 [==============================] - 27s 14ms/step - loss: 1.3742e-06 - accuracy: 0.0984
Epoch 4/10
1875/1875 [==============================] - 19s 10ms/step - loss: 1.0828e-06 - accuracy: 0.0984
Epoch 5/10
1875/1875 [==============================] - 25s 13ms/step - loss: 9.9699e-07 - accuracy: 0.0993
Epoch 6/10
1875/1875 [==============================] - 22s 12ms/step - loss: 9.7028e-07 - accuracy: 0.0989
Epoch 7/10
1875/1875 [==============================] - 20s 11ms/step - loss: 9.6041e-07 - accuracy: 0.0978
Epoch 8/10
1875/1875 [==============================] - 27s 14ms/step - loss: 9.5795e-07 - accuracy: 0.0973
Epoch 9/10
1875/1875 [==============================] - 23s 13ms/step - loss: 9.5548e-07 - accuracy: 0.0960
Epoch 10/10
1875/1875 [====================

In [16]:
model.evaluate(ds_test, steps=10)

10/10 [==============================] - 1s 56ms/step - loss: 9.5367e-07 - accuracy: 0.0656


[9.536738616588991e-07, 0.065625]

In [17]:
# Usage of Callback

import datetime as dt

callbacks = [
  # Write TensorBoard logs to log_dir directory
  tf.keras.callbacks.TensorBoard(log_dir='./tflog/{}'.format(dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")))
]

In [ ]:
model.fit(ds_train, epochs=epochs, steps_per_epoch=steps_per_epoch,
          validation_data=ds_test,
          validation_steps=3, callbacks=callbacks)

Epoch 1/100
1875/1875 [==============================] - 35s 19ms/step - loss: 9.5387e-07 - accuracy: 0.0987 - val_loss: 9.5367e-07 - val_accuracy: 0.0729
Epoch 2/100
1875/1875 [==============================] - 44s 23ms/step - loss: 9.5391e-07 - accuracy: 0.0987 - val_loss: 9.5367e-07 - val_accuracy: 0.0729
Epoch 3/100
1875/1875 [==============================] - 44s 24ms/step - loss: 9.5398e-07 - accuracy: 0.0987 - val_loss: 9.5367e-07 - val_accuracy: 0.0729
Epoch 4/100
1875/1875 [==============================] - 36s 19ms/step - loss: 9.5368e-07 - accuracy: 0.0987 - val_loss: 9.5367e-07 - val_accuracy: 0.0729
Epoch 5/100
1875/1875 [==============================] - 38s 20ms/step - loss: 9.5373e-07 - accuracy: 0.0987 - val_loss: 9.5367e-07 - val_accuracy: 0.0729
Epoch 6/100
1875/1875 [==============================] - 49s 26ms/step - loss: 9.5369e-07 - accuracy: 0.0987 - val_loss: 9.5367e-07 - val_accuracy: 0.0729
Epoch 7/100
1875/1875 [==============================] - 50s 27ms/step

In [ ]:
model.evaluate(ds_test,steps=10)

In [19]:
model.evaluate(ds_test,steps=10)

10/10 [==============================] - 0s 19ms/step - loss: 0.0000e+00 - accuracy: 0.1156


[0.0, 0.115625]